## ToDo: Set a standard date range for all charts


### Which sector has highest impact on CO2 emissions?

TODO: what does each sector mean? describe

In [30]:
import pandas as pd 
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
data = pd.read_csv("data/ghg-emissions.csv")

In [56]:
# Use the aggregated data from the bar chart
bar_data = data.groupby('Sector').sum().reset_index()

# Use absolute values for the pie chart to include all categories
bar_data['Emissions'] = bar_data['Emissions'].abs()

fig_pie = px.pie(bar_data, 
                 values='Emissions', 
                 names='Sector', 
                 title='Sectoral Distribution of Emissions',
                 hover_data={'Emissions': ':.2f'})
fig_pie.update_layout(title_x=0.5)
fig_pie.show()


In [32]:
def create_sector_emissions_animation(data_df):
    # Prepare data
    data = data_df.groupby(['Year', 'Sector'])['Emissions'].sum().reset_index()
    unique_sectors = data['Sector'].unique()
    
    # Skip frames: Use every nth year for faster animations
    n = 2
    data = data[data['Year'] % n == 0]

    # Create frames for animation
    frames = []
    for year in sorted(data['Year'].unique()):
        frame_data = []
        for sector in unique_sectors:
            sector_data = data[(data['Year'] == year) & (data['Sector'] == sector)]
            frame_data.append(go.Bar(
                y=[sector],  # Single sector name
                x=sector_data["Emissions"].values if not sector_data.empty else [0],
                orientation='h',
                name=sector,
            ))
        
        # Calculate dynamic axis range
        min_x = data[data['Year'] <= year]['Emissions'].min()
        max_x = data[data['Year'] <= year]['Emissions'].max()
        
        frames.append(go.Frame(
            data=frame_data,
            name=str(year),
            layout=go.Layout(
                xaxis={"range": [min_x - 10, max_x + 10]}
            )
        ))

    # Slider steps
    slider_steps = []
    for year in sorted(data['Year'].unique()):
        slider_step = {
            "method": "animate",
            "args": [
                [str(year)],
                {
                    "frame": {"duration": 100, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 0}
                }
            ],
            "label": str(year)
        }
        slider_steps.append(slider_step)

    # Create the base figure
    base_data = []
    for sector in unique_sectors:
        sector_data = data[(data['Year'] == data['Year'].min()) & (data['Sector'] == sector)]
        base_data.append(go.Bar(
            y=[sector],
            x=sector_data["Emissions"].values if not sector_data.empty else [0],
            orientation='h',
            name=sector,
        ))

    fig = go.Figure(
        data=base_data,
        layout=go.Layout(
            title="Sector-Wise CO2 Emissions Over Time (Animated with Dynamic Axes)",
            yaxis=dict(title="Sector"),
            xaxis=dict(title="Emissions (MtCO2e)", range=[data['Emissions'].min() - 10, data['Emissions'].max() + 10]),
            width=950,
            height=600,
            updatemenus=[{
                "buttons": [
                    {
                        "args": [None, {
                            "frame": {"duration": 100, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 0}
                        }],
                        "label": "Play",
                        "method": "animate"
                    },
                    {
                        "args": [[None], {
                            "frame": {"duration": 0, "redraw": False},
                            "mode": "immediate",
                            "transition": {"duration": 0}
                        }],
                        "label": "Pause",
                        "method": "animate"
                    }
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top"
            }],
            sliders=[{
                "active": 0,
                "yanchor": "top",
                "xanchor": "left",
                "currentvalue": {
                    "font": {"size": 20},
                    "prefix": "Year: ",
                    "visible": True,
                    "xanchor": "right"
                },
                "transition": {"duration": 300, "easing": "cubic-in-out"},
                "pad": {"b": 10, "t": 50},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": slider_steps
            }]
        ),
        frames=frames
    )

    return fig

fig = create_sector_emissions_animation(data)
fig.show()



We can observe that Energy Production is the single biggest contributor to the CO2 emissions. The other sectors like Industrial Processes, Agriculture, and Waste also contribute to the CO2 emissions but the impact is not as high Energy Production.
- Land-Use Change and Forestry has a negative impact on CO2 emissions. This means that the sector is absorbing more CO2 than it is emitting. This draws to our next question, which is, which countries are contributing to the land use and forestry?

### CO2 Emission from Land-Use Change and Forestry

In [72]:
# Read the CSV file
df_land_use = pd.read_csv('data/co2-land-use.csv')

# Create animated choropleth map
fig_land_use = px.choropleth(df_land_use,
                             locations="Code",  # Country codes
                             locationmode="ISO-3",
                             color="Annual CO2 emissions from land-use change",
                             hover_name="Entity",
                             animation_frame="Year",
                             # use a blue and green color scale
                             color_continuous_scale=["blue", "white", "red"],
                             title="Annual CO2 Emissions from Land-Use Change (1850-2023)",
                             range_color=(-100_000_000, 100_000_000))

# Update layout
fig_land_use.update_layout(
    title_x=0.5,
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='equirectangular'
    ),
    width=1000,
    height=600
)

# Set opacity
fig_land_use.update_traces(marker=dict(opacity=0.6))

# Add play and pause button settings
fig_land_use.update_layout(
    updatemenus=[{
        'type': 'buttons',
        'showactive': False,
        'buttons': [
            {
                'label': 'Play',
                'method': 'animate',
                'args': [None, {
                    'frame': {'duration': 10, 'redraw': True},
                    'fromcurrent': True,
                    'transition': {'duration': 0}
                }]
            },
            {
                'label': 'Pause',
                'method': 'animate',
                'args': [[None], {
                    'frame': {'duration': 0, 'redraw': False},
                    'mode': 'immediate',
                    'transition': {'duration': 0}
                }]
            }
        ]
    }]
)

# Show plot
fig_land_use.show()


- We can observe that Arab Countries like Saudi Arabia, Iraq, and Iran are consistently contributed the least to the CO2 emissions from Land-Use Change and Forestry. This is interesting because these countries are known for their oil production and transforming landscapes. 
- China, and India are among the biggest contributors to the CO2 emissions from Land-Use Change and Forestry. This is interesting because these countries are known for their rapid industrialization and urbanization.

### Is there any relationship between population and CO2 Emissions?

In [73]:
# Import required libraries
import pandas as pd
import plotly.express as px

# Read the CSV file
df = pd.read_csv('data/share-co2-emissions-vs-population.csv')

# Clean data - remove rows with missing values
df = df.dropna()

# Create animated scatter plot
fig = px.scatter(df,
                 x="Share of world population",
                 y="Share of global annual CO₂ emissions",
                 animation_frame="Year",
                 animation_group="Entity",
                 size="Share of global annual CO₂ emissions",
                 color="World Bank's income classification",
                 hover_name="Entity",
                 log_x=True,
                 log_y=True,
                 size_max=100,  # Increased size of points
                 title="CO2 Emissions vs Population Share by Country (1949-1988)")

# Update layout to match the x-axis and y-axis scale and make the canvas larger
fig.update_layout(
    xaxis=dict(
        title="Share of World Population",
        type="log",
        range=[-3, 3],  # Log scale for 0.001% (10^-3) to 10% (10^3)
        tickvals=[0.001, 0.01, 0.1, 1, 10],
        ticktext=["0.001%", "0.01%", "0.1%", "1%", "10%"]
    ),
    yaxis=dict(
        title="Share of Global CO₂ Emissions",
        type="log",
        range=[-3, 3],  # Log scale for 0.001% (10^-3) to 10% (10^3)
        tickvals=[0.001, 0.01, 0.1, 1, 10],
        ticktext=["0.001%", "0.01%", "0.1%", "1%", "10%"]
    ),
    width=1000,  # Increased canvas width
    height=700,  # Increased canvas height
    showlegend=True
)

# Add play and pause button animation settings
fig.update_layout(
    updatemenus=[{
        'type': 'buttons',
        'showactive': True,
        'buttons': [
            {
                'label': 'Play',
                'method': 'animate',
                'args': [None, {'frame': {'duration': 200, 'redraw': False}, 'fromcurrent': True}]
            },
            {
                'label': 'Pause',
                'method': 'animate',
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}]
            }
        ]
    }]
)

# Show plot
fig.show()

# # Optional: Save as HTML file
# fig.write_html("co2_vs_population_animation.html")


- We can observe an intersting linear relationship between Global CO2 Emissions and the World Population.
- Higher the population, higher their CO2 emissions, which makes sense because more people means more consumption of resources and more waste generation.